[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/danpele/Time-Series-Analysis/blob/main/chapter3_seminar_notebook.ipynb)

---

# Chapter 3 Seminar: ARIMA Models - Practice Exercises

**Course:** Time Series Analysis and Forecasting  
**Program:** Bachelor program, Faculty of Cybernetics, Statistics and Economic Informatics, Bucharest University of Economic Studies, Romania  
**Academic Year:** 2025-2026

---

## Seminar Objectives

1. Practice unit root testing with real data
2. Apply the Box-Jenkins methodology
3. Fit and diagnose ARIMA models
4. Generate and evaluate forecasts
5. Work with financial and macroeconomic time series

## Setup

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Time series
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller, kpss, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
from scipy import stats
import yfinance as yf

# Auto-ARIMA
try:
    import pmdarima as pm
except:
    !pip install pmdarima -q
    import pmdarima as pm

# Plotting style
plt.rcParams['figure.figsize'] = (12, 5)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.facecolor'] = 'none'
plt.rcParams['figure.facecolor'] = 'none'
plt.rcParams['savefig.facecolor'] = 'none'
plt.rcParams['savefig.transparent'] = True
plt.rcParams['legend.frameon'] = False
plt.rcParams['axes.grid'] = False
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False

COLORS = {'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32', 'orange': '#E67E22'}

print("Setup complete!")

# Part 1: Multiple Choice Quiz

Test your understanding of ARIMA models, unit roots, and integration concepts. For each question, assign your answer (A, B, C, or D) to the variable `answer` and run the cell to check.

### Quiz 1: Unit Root Definition

A unit root in a time series means:

A) The series has zero mean  
B) The autoregressive polynomial has a root equal to 1  
C) The series is always positive  
D) The variance is constant over time

In [ ]:
# Quiz 1: Enter your answer (A, B, C, or D)
answer = ""  # Replace with your answer

# Check answer
if answer.upper() == "B":
    print("Correct! A unit root means the AR polynomial (1 - phi*L) = 0 has a root at L=1,")
    print("which implies the coefficient phi = 1, making the process non-stationary.")
elif answer.upper() in ["A", "C", "D"]:
    print("Incorrect. Think about what makes a process like Y_t = Y_{t-1} + e_t non-stationary.")
else:
    print("Please enter A, B, C, or D")

### Quiz 2: Differencing Operator

The first difference operator applied to a series Y_t is defined as:

A) Y_t + Y_{t-1}  
B) Y_t - Y_{t-1}  
C) Y_t / Y_{t-1}  
D) Y_t * Y_{t-1}

In [ ]:
# Quiz 2: Enter your answer (A, B, C, or D)
answer = ""  # Replace with your answer

# Check answer
if answer.upper() == "B":
    print("Correct! The first difference is ΔY_t = Y_t - Y_{t-1} = (1-L)Y_t,")
    print("where L is the lag operator. This removes unit roots and trends.")
elif answer.upper() in ["A", "C", "D"]:
    print("Incorrect. The difference operator subtracts consecutive values.")
else:
    print("Please enter A, B, C, or D")

### Quiz 3: ARIMA Notation

In an ARIMA(1,1,2) model, what do the numbers represent?

A) 1 MA term, 1 difference, 2 AR terms  
B) 1 AR term, 1 difference, 2 MA terms  
C) 1 seasonal AR term, 1 trend, 2 seasonal MA terms  
D) 1 lag, 1 forecast horizon, 2 parameters

In [ ]:
# Quiz 3: Enter your answer (A, B, C, or D)
answer = ""  # Replace with your answer

# Check answer
if answer.upper() == "B":
    print("Correct! ARIMA(p,d,q) means p AR terms, d differences, q MA terms.")
    print("So ARIMA(1,1,2) has 1 AR term, is differenced once, and has 2 MA terms.")
elif answer.upper() in ["A", "C", "D"]:
    print("Incorrect. Remember: ARIMA(p,d,q) where p=AR order, d=differencing, q=MA order.")
else:
    print("Please enter A, B, C, or D")

### Quiz 4: I(1) vs I(2) Processes

A series that requires two differences to become stationary is called:

A) I(0)  
B) I(1)  
C) I(2)  
D) I(1/2)

In [ ]:
# Quiz 4: Enter your answer (A, B, C, or D)
answer = ""  # Replace with your answer

# Check answer
if answer.upper() == "C":
    print("Correct! I(d) denotes the order of integration - the number of differences")
    print("needed for stationarity. I(2) means two differences are required.")
elif answer.upper() in ["A", "B", "D"]:
    print("Incorrect. The number in I(d) indicates how many times you must difference.")
else:
    print("Please enter A, B, C, or D")

### Quiz 5: ADF Test Interpretation

In the Augmented Dickey-Fuller (ADF) test, a p-value of 0.02 suggests:

A) The series has a unit root (non-stationary)  
B) The series is stationary (reject unit root hypothesis)  
C) The test is inconclusive  
D) More differencing is needed

In [ ]:
# Quiz 5: Enter your answer (A, B, C, or D)
answer = ""  # Replace with your answer

# Check answer
if answer.upper() == "B":
    print("Correct! ADF null hypothesis is 'unit root exists'. A small p-value (< 0.05)")
    print("means we reject H0 and conclude the series is stationary.")
elif answer.upper() in ["A", "C", "D"]:
    print("Incorrect. Remember: ADF tests H0: unit root. Small p-value = reject H0 = stationary.")
else:
    print("Please enter A, B, C, or D")

### Quiz 6: KPSS Test Interpretation

The KPSS test differs from the ADF test because:

A) KPSS has stationarity as the null hypothesis  
B) KPSS tests for seasonality  
C) KPSS only works with monthly data  
D) KPSS requires normally distributed errors

In [ ]:
# Quiz 6: Enter your answer (A, B, C, or D)
answer = ""  # Replace with your answer

# Check answer
if answer.upper() == "A":
    print("Correct! KPSS tests H0: stationary vs H1: unit root.")
    print("This is opposite to ADF. Using both tests together provides more robust conclusions.")
elif answer.upper() in ["B", "C", "D"]:
    print("Incorrect. The key difference is in the null hypothesis formulation.")
else:
    print("Please enter A, B, C, or D")

### Quiz 7: Random Walk with Drift

A random walk with drift is characterized by:

A) Y_t = c + Y_{t-1} + e_t where c is a constant  
B) Y_t = phi * Y_{t-1} + e_t where |phi| < 1  
C) Y_t = e_t (white noise)  
D) Y_t = c + beta*t + e_t (deterministic trend)

In [ ]:
# Quiz 7: Enter your answer (A, B, C, or D)
answer = ""  # Replace with your answer

# Check answer
if answer.upper() == "A":
    print("Correct! A random walk with drift has the form Y_t = c + Y_{t-1} + e_t.")
    print("The drift term 'c' causes the series to trend upward (c>0) or downward (c<0).")
elif answer.upper() in ["B", "C", "D"]:
    print("Incorrect. A random walk with drift adds a constant to the basic random walk.")
else:
    print("Please enter A, B, C, or D")

### Quiz 8: Over-differencing Signs

What is a sign that a series has been over-differenced?

A) The ACF shows slow decay  
B) The ACF at lag 1 is strongly negative (close to -0.5 or below)  
C) The series still has a visible trend  
D) The ADF test still rejects stationarity

In [ ]:
# Quiz 8: Enter your answer (A, B, C, or D)
answer = ""  # Replace with your answer

# Check answer
if answer.upper() == "B":
    print("Correct! Over-differencing introduces artificial negative autocorrelation.")
    print("A strongly negative ACF(1) near -0.5 is a classic symptom of over-differencing.")
elif answer.upper() in ["A", "C", "D"]:
    print("Incorrect. Over-differencing creates artificial patterns, especially in the ACF.")
else:
    print("Please enter A, B, C, or D")

### Quiz 9: Trend Stationarity vs Difference Stationarity

The key difference between trend stationary and difference stationary processes is:

A) Trend stationary processes have no trend  
B) Difference stationary processes can be made stationary by removing a deterministic trend  
C) Trend stationary processes become stationary by detrending, difference stationary by differencing  
D) They are the same thing with different names

In [ ]:
# Quiz 9: Enter your answer (A, B, C, or D)
answer = ""  # Replace with your answer

# Check answer
if answer.upper() == "C":
    print("Correct! Trend stationary: Y_t = a + bt + e_t (remove trend by regression).")
    print("Difference stationary: Y_t = Y_{t-1} + e_t (remove unit root by differencing).")
elif answer.upper() in ["A", "B", "D"]:
    print("Incorrect. The distinction is about how to achieve stationarity.")
else:
    print("Please enter A, B, C, or D")

### Quiz 10: Order of Integration

If a series Y_t is I(1) and we compute Z_t = Y_t - Y_{t-1}, then Z_t is:

A) I(2)  
B) I(1)  
C) I(0)  
D) Still non-stationary

In [ ]:
# Quiz 10: Enter your answer (A, B, C, or D)
answer = ""  # Replace with your answer

# Check answer
if answer.upper() == "C":
    print("Correct! Differencing an I(1) series once yields an I(0) (stationary) series.")
    print("The order of integration decreases by 1 with each difference: I(d) -> I(d-1).")
elif answer.upper() in ["A", "B", "D"]:
    print("Incorrect. Differencing reduces the order of integration by 1.")
else:
    print("Please enter A, B, C, or D")

### Quiz 11: Cointegration Basics

Two I(1) series X_t and Y_t are cointegrated if:

A) Both become stationary after differencing  
B) A linear combination of them is I(0) (stationary)  
C) They have the same variance  
D) Their correlation is exactly 1

In [ ]:
# Quiz 11: Enter your answer (A, B, C, or D)
answer = ""  # Replace with your answer

# Check answer
if answer.upper() == "B":
    print("Correct! Cointegration means Y_t - beta*X_t = e_t where e_t is I(0).")
    print("The series share a common stochastic trend and have a long-run equilibrium.")
elif answer.upper() in ["A", "C", "D"]:
    print("Incorrect. Cointegration is about finding a stationary linear combination.")
else:
    print("Please enter A, B, C, or D")

### Quiz 12: Spurious Regression

Spurious regression occurs when:

A) We regress one stationary series on another  
B) We regress one I(1) series on another unrelated I(1) series and get significant results  
C) We use too many lagged variables  
D) The residuals are heteroskedastic

In [ ]:
# Quiz 12: Enter your answer (A, B, C, or D)
answer = ""  # Replace with your answer

# Check answer
if answer.upper() == "B":
    print("Correct! Spurious regression: high R-squared and significant t-stats between")
    print("unrelated I(1) series. Standard inference fails with non-stationary data.")
elif answer.upper() in ["A", "C", "D"]:
    print("Incorrect. Spurious regression is a problem specific to non-stationary series.")
else:
    print("Please enter A, B, C, or D")

### Quiz 13: Phillips-Perron Test

The Phillips-Perron test differs from the ADF test primarily in how it:

A) Handles serial correlation and heteroskedasticity  
B) Selects the number of lags  
C) Specifies the null hypothesis  
D) Computes critical values

In [ ]:
# Quiz 13: Enter your answer (A, B, C, or D)
answer = ""  # Replace with your answer

# Check answer
if answer.upper() == "A":
    print("Correct! PP test uses non-parametric corrections for serial correlation")
    print("and heteroskedasticity, while ADF adds lagged differences to the regression.")
elif answer.upper() in ["B", "C", "D"]:
    print("Incorrect. The main difference is in handling nuisance parameters.")
else:
    print("Please enter A, B, C, or D")

### Quiz 14: Structural Breaks

In the presence of a structural break, unit root tests may:

A) Become more powerful  
B) Fail to reject the unit root null even when the series is stationary  
C) Always correctly identify stationarity  
D) Produce negative test statistics

In [ ]:
# Quiz 14: Enter your answer (A, B, C, or D)
answer = ""  # Replace with your answer

# Check answer
if answer.upper() == "B":
    print("Correct! Structural breaks can make standard unit root tests biased toward")
    print("non-rejection. Tests like Zivot-Andrews account for unknown break points.")
elif answer.upper() in ["A", "C", "D"]:
    print("Incorrect. Structural breaks reduce the power of standard unit root tests.")
else:
    print("Please enter A, B, C, or D")

### Quiz 15: Deterministic vs Stochastic Trends

A stochastic trend differs from a deterministic trend because:

A) Stochastic trends are caused by shocks that have permanent effects  
B) Deterministic trends cannot be removed  
C) Stochastic trends follow a straight line  
D) Deterministic trends are unpredictable

In [ ]:
# Quiz 15: Enter your answer (A, B, C, or D)
answer = ""  # Replace with your answer

# Check answer
if answer.upper() == "A":
    print("Correct! In a stochastic trend (unit root), shocks accumulate permanently.")
    print("Deterministic trends are predictable functions of time (e.g., Y_t = a + bt + e_t).")
elif answer.upper() in ["B", "C", "D"]:
    print("Incorrect. The key is whether shocks have temporary or permanent effects.")
else:
    print("Please enter A, B, C, or D")

### Quiz 16: ARIMA Forecasting

For an ARIMA(0,1,0) model (random walk), the optimal h-step ahead forecast is:

A) Zero  
B) The last observed value  
C) The sample mean  
D) A linear extrapolation of recent values

In [ ]:
# Quiz 16: Enter your answer (A, B, C, or D)
answer = ""  # Replace with your answer

# Check answer
if answer.upper() == "B":
    print("Correct! For a random walk, E[Y_{t+h}|Y_t] = Y_t for all h.")
    print("The best forecast is always the last observed value (no-change forecast).")
elif answer.upper() in ["A", "C", "D"]:
    print("Incorrect. Think about what happens when you iterate Y_{t+1} = Y_t + e_{t+1}.")
else:
    print("Please enter A, B, C, or D")

### Quiz 17: Differencing and Variance

When you difference an I(1) series, what typically happens to the variance?

A) Variance always increases  
B) Variance always decreases  
C) Variance can increase or decrease depending on the series  
D) Variance becomes exactly zero

In [ ]:
# Quiz 17: Enter your answer (A, B, C, or D)
answer = ""  # Replace with your answer

# Check answer
if answer.upper() == "C":
    print("Correct! Differencing removes the unit root but changes variance in complex ways.")
    print("For pure random walks, Var(ΔY) = Var(e). The effect depends on the DGP.")
elif answer.upper() in ["A", "B", "D"]:
    print("Incorrect. The variance change depends on the underlying process structure.")
else:
    print("Please enter A, B, C, or D")

### Quiz 18: Unit Root in AR Polynomial

For the AR(1) process Y_t = phi*Y_{t-1} + e_t, a unit root exists when:

A) phi = 0  
B) phi = 1  
C) phi = -1  
D) phi = 0.5

In [ ]:
# Quiz 18: Enter your answer (A, B, C, or D)
answer = ""  # Replace with your answer

# Check answer
if answer.upper() == "B":
    print("Correct! When phi = 1, the AR polynomial (1 - phi*L) = (1 - L) has root L = 1.")
    print("This makes Y_t = Y_{t-1} + e_t, a random walk (unit root process).")
elif answer.upper() in ["A", "C", "D"]:
    print("Incorrect. A unit root means the coefficient on Y_{t-1} equals exactly 1.")
else:
    print("Please enter A, B, C, or D")

### Quiz 19: Long-run Forecast Convergence

For a stationary ARMA process, long-run forecasts converge to:

A) Zero  
B) The last observed value  
C) The unconditional mean of the process  
D) Infinity

In [ ]:
# Quiz 19: Enter your answer (A, B, C, or D)
answer = ""  # Replace with your answer

# Check answer
if answer.upper() == "C":
    print("Correct! For stationary ARMA, as h -> infinity, forecasts revert to the mean.")
    print("This is mean reversion - a key property distinguishing stationary from unit root.")
elif answer.upper() in ["A", "B", "D"]:
    print("Incorrect. Stationary processes have finite unconditional means they revert to.")
else:
    print("Please enter A, B, C, or D")

### Quiz 20: Model Identification Steps

In the Box-Jenkins methodology, the correct order of steps is:

A) Estimation, Identification, Diagnostic checking, Forecasting  
B) Identification, Estimation, Diagnostic checking, Forecasting  
C) Forecasting, Estimation, Identification, Diagnostic checking  
D) Diagnostic checking, Identification, Estimation, Forecasting

In [ ]:
# Quiz 20: Enter your answer (A, B, C, or D)
answer = ""  # Replace with your answer

# Check answer
if answer.upper() == "B":
    print("Correct! Box-Jenkins: 1) Identify (p,d,q) using ACF/PACF,")
    print("2) Estimate parameters, 3) Check residual diagnostics, 4) Forecast if adequate.")
elif answer.upper() in ["A", "C", "D"]:
    print("Incorrect. You must identify the model structure before you can estimate it.")
else:
    print("Please enter A, B, C, or D")

## Exercise 1: Unit Root Testing

### Task
Download stock price data and determine if it's stationary. Test both prices and returns.

In [ ]:
# Download Apple stock data
ticker = "AAPL"
data = yf.download(ticker, start='2020-01-01', end='2024-12-31', progress=False)

# Handle MultiIndex columns
if isinstance(data.columns, pd.MultiIndex):
    data.columns = data.columns.droplevel(1)

prices = data['Close']
returns = prices.pct_change().dropna() * 100  # Percentage returns

print(f"Downloaded {ticker} data: {len(prices)} observations")
print(f"Period: {prices.index[0].date()} to {prices.index[-1].date()}")

In [ ]:
# Plot prices and returns
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

axes[0].plot(prices.index, prices.values, color=COLORS['blue'], linewidth=1, label='Price')
axes[0].set_title(f'{ticker} Stock Price', fontweight='bold')
axes[0].set_ylabel('Price ($)')

axes[1].plot(returns.index, returns.values, color=COLORS['green'], linewidth=0.5, label='Returns')
axes[1].axhline(y=0, color='red', linestyle='--', alpha=0.5)
axes[1].set_title(f'{ticker} Daily Returns (%)', fontweight='bold')
axes[1].set_ylabel('Return (%)')

# Collect handles and labels
handles, labels = [], []
for ax in axes:
    h, l = ax.get_legend_handles_labels()
    handles.extend(h)
    labels.extend(l)

plt.subplots_adjust(bottom=0.12)
fig.legend(handles, labels, loc='lower center', ncol=2, bbox_to_anchor=(0.5, -0.02))
plt.tight_layout(rect=[0, 0.05, 1, 1])
plt.show()

In [ ]:
def comprehensive_unit_root_test(series, name):
    """Run ADF and KPSS tests with interpretation."""
    print(f"\n{'='*60}")
    print(f"Unit Root Tests: {name}")
    print('='*60)
    
    # ADF Test
    adf = adfuller(series.dropna(), autolag='AIC')
    print(f"\nADF Test (H0: Unit root exists)")
    print(f"  Statistic: {adf[0]:.4f}")
    print(f"  p-value: {adf[1]:.6f}")
    print(f"  Lags used: {adf[2]}")
    adf_result = "Stationary" if adf[1] < 0.05 else "Non-stationary"
    print(f"  Result: {adf_result}")
    
    # KPSS Test
    kpss_stat = kpss(series.dropna(), regression='c', nlags='auto')
    print(f"\nKPSS Test (H0: Stationary)")
    print(f"  Statistic: {kpss_stat[0]:.4f}")
    print(f"  p-value: {kpss_stat[1]:.4f}")
    kpss_result = "Stationary" if kpss_stat[1] > 0.05 else "Non-stationary"
    print(f"  Result: {kpss_result}")
    
    # Combined interpretation
    print(f"\nCombined Conclusion:")
    if adf_result == "Stationary" and kpss_result == "Stationary":
        print("  Both tests agree: STATIONARY")
    elif adf_result == "Non-stationary" and kpss_result == "Non-stationary":
        print("  Both tests agree: NON-STATIONARY")
    else:
        print(f"  Conflicting results: ADF={adf_result}, KPSS={kpss_result}")
    
    return adf_result, kpss_result

In [ ]:
# Test prices
comprehensive_unit_root_test(prices, f"{ticker} Prices")

In [ ]:
# Test returns
comprehensive_unit_root_test(returns, f"{ticker} Returns")

### Exercise 1 Questions

1. What is the order of integration for stock prices?
2. Why are returns stationary while prices are not?
3. What economic interpretation does this have?

## Exercise 2: ACF/PACF Analysis

### Task
Analyze ACF and PACF patterns to identify appropriate model orders.

In [ ]:
# ACF/PACF of prices vs returns
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# Prices
plot_acf(prices.dropna(), ax=axes[0, 0], lags=30, color=COLORS['blue'])
axes[0, 0].set_title('ACF: Stock Prices (slow decay = non-stationary)', fontweight='bold')

plot_pacf(prices.dropna(), ax=axes[0, 1], lags=30, color=COLORS['blue'], method='ywm')
axes[0, 1].set_title('PACF: Stock Prices', fontweight='bold')

# Returns
plot_acf(returns.dropna(), ax=axes[1, 0], lags=30, color=COLORS['green'])
axes[1, 0].set_title('ACF: Returns (near zero = stationary)', fontweight='bold')

plot_pacf(returns.dropna(), ax=axes[1, 1], lags=30, color=COLORS['green'], method='ywm')
axes[1, 1].set_title('PACF: Returns', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
print("Observation: Prices show slow ACF decay (unit root), returns are nearly white noise")

## Exercise 3: Box-Jenkins Methodology with Real Data

### Task
Apply the complete Box-Jenkins procedure to US Industrial Production data.

**Steps:**
1. Plot data and check for trends
2. Test for unit roots
3. Difference if necessary
4. Identify p, q from ACF/PACF
5. Estimate model
6. Diagnose residuals
7. Forecast

In [ ]:
# Step 1: Plot the data
fig, ax = plt.subplots(figsize=(14, 5))
ax.plot(range(len(y_train)), y_train, color=COLORS['blue'], linewidth=1, label='Training')
ax.plot(range(len(y_train), len(y_full)), y_test, color=COLORS['orange'], linewidth=1, label='Test')
ax.axvline(x=train_size, color='black', linestyle='--', alpha=0.5)
ax.set_title('Step 1: US Industrial Production Index', fontweight='bold')
ax.set_xlabel('Time')
ax.set_ylabel('Index')

# Get handles and labels for fig.legend
handles, labels = ax.get_legend_handles_labels()

plt.subplots_adjust(bottom=0.15)
fig.legend(handles, labels, loc='lower center', ncol=2, bbox_to_anchor=(0.5, -0.02))
plt.tight_layout(rect=[0, 0.08, 1, 1])
plt.show()

In [ ]:
print("Observation: Series shows an upward trend with some volatility (non-stationary)")

In [ ]:
# Step 3: First difference
diff_train = np.diff(y_train)

fig, axes = plt.subplots(1, 2, figsize=(14, 4))

axes[0].plot(diff_train, color=COLORS['green'], linewidth=0.8, label='ΔY_t')
axes[0].axhline(y=0, color='red', linestyle='--', alpha=0.5)
axes[0].set_title('Step 3: First Differenced Series', fontweight='bold')

# Test differenced series
adf_diff = adfuller(diff_train)
axes[1].text(0.5, 0.7, f"ADF on ΔY_t:", transform=axes[1].transAxes, fontsize=14, fontweight='bold', ha='center')
axes[1].text(0.5, 0.5, f"Statistic: {adf_diff[0]:.4f}", transform=axes[1].transAxes, fontsize=12, ha='center')
axes[1].text(0.5, 0.35, f"p-value: {adf_diff[1]:.6f}", transform=axes[1].transAxes, fontsize=12, ha='center')
result = "STATIONARY" if adf_diff[1] < 0.05 else "NON-STATIONARY"
axes[1].text(0.5, 0.2, f"Result: {result}", transform=axes[1].transAxes, fontsize=12, fontweight='bold', ha='center',
            color=COLORS['green'] if result=='STATIONARY' else COLORS['red'])
axes[1].axis('off')
axes[1].set_title('Unit Root Test on Differences', fontweight='bold')

# Get handles and labels for fig.legend
handles, labels = axes[0].get_legend_handles_labels()

plt.subplots_adjust(bottom=0.18)
fig.legend(handles, labels, loc='lower center', ncol=1, bbox_to_anchor=(0.5, -0.02))
plt.tight_layout(rect=[0, 0.08, 1, 1])
plt.show()

In [ ]:
# Step 4: ACF/PACF of differenced series
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

plot_acf(diff_train, ax=axes[0], lags=25, color=COLORS['blue'])
axes[0].set_title('Step 4a: ACF of ΔY_t', fontweight='bold')

plot_pacf(diff_train, ax=axes[1], lags=25, color=COLORS['blue'], method='ywm')
axes[1].set_title('Step 4b: PACF of ΔY_t', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
print("Identification:")
print("- ACF: Decays (suggests MA component or ARMA)")
print("- PACF: Decays (suggests AR component or ARMA)")
print("- Both decay → ARMA structure in differences → ARIMA")

In [ ]:
# Step 2: Unit root test
comprehensive_unit_root_test(pd.Series(y_train), "Training Data")

In [ ]:
# Step 3: First difference
diff_train = np.diff(y_train)

fig, axes = plt.subplots(1, 2, figsize=(14, 4))

axes[0].plot(diff_train, color=COLORS['green'], linewidth=0.8, label='ΔY_t')
axes[0].axhline(y=0, color='red', linestyle='--', alpha=0.5)
axes[0].set_title('Step 3: First Differenced Series', fontweight='bold')
axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.12), frameon=False)

# Test differenced series
adf_diff = adfuller(diff_train)
axes[1].text(0.5, 0.7, f"ADF on ΔY_t:", transform=axes[1].transAxes, fontsize=14, fontweight='bold', ha='center')
axes[1].text(0.5, 0.5, f"Statistic: {adf_diff[0]:.4f}", transform=axes[1].transAxes, fontsize=12, ha='center')
axes[1].text(0.5, 0.35, f"p-value: {adf_diff[1]:.6f}", transform=axes[1].transAxes, fontsize=12, ha='center')
result = "STATIONARY" if adf_diff[1] < 0.05 else "NON-STATIONARY"
axes[1].text(0.5, 0.2, f"Result: {result}", transform=axes[1].transAxes, fontsize=12, fontweight='bold', ha='center',
            color=COLORS['green'] if result=='STATIONARY' else COLORS['red'])
axes[1].axis('off')
axes[1].set_title('Unit Root Test on Differences', fontweight='bold')

plt.tight_layout()
plt.show()

print(f"Conclusion: d = 1 (one difference makes the series stationary)")

In [ ]:
# Step 6: Diagnostic checking
residuals = best_model.resid

fig, axes = plt.subplots(2, 2, figsize=(14, 8))
fig.suptitle('Step 6: Residual Diagnostics', fontsize=14, fontweight='bold')

# Residuals over time
axes[0, 0].plot(residuals, color=COLORS['blue'], linewidth=0.5, label='Residuals')
axes[0, 0].axhline(y=0, color='red', linestyle='--')
axes[0, 0].set_title('Residuals Over Time', fontweight='bold')

# Histogram
axes[0, 1].hist(residuals, bins=30, color=COLORS['blue'], edgecolor='black', 
                alpha=0.7, density=True, label='Residuals')
x = np.linspace(residuals.min(), residuals.max(), 100)
axes[0, 1].plot(x, stats.norm.pdf(x, residuals.mean(), residuals.std()), 
                color=COLORS['red'], linewidth=2, label='Normal')
axes[0, 1].set_title('Residual Distribution', fontweight='bold')

# ACF of residuals
plot_acf(residuals, ax=axes[1, 0], lags=20, color=COLORS['blue'])
axes[1, 0].set_title('ACF of Residuals (should be white noise)', fontweight='bold')

# Q-Q plot
(osm, osr), (slope, intercept, r) = stats.probplot(residuals, dist="norm")
axes[1, 1].scatter(osm, osr, color=COLORS['blue'], s=20, alpha=0.5, label='Sample')
axes[1, 1].plot(osm, slope*osm + intercept, color=COLORS['red'], linewidth=2, label='Theoretical')
axes[1, 1].set_title('Q-Q Plot', fontweight='bold')

# Collect handles and labels from relevant subplots
handles, labels = [], []
for ax in [axes[0, 0], axes[0, 1], axes[1, 1]]:
    h, l = ax.get_legend_handles_labels()
    handles.extend(h)
    labels.extend(l)

plt.subplots_adjust(bottom=0.12)
fig.legend(handles, labels, loc='lower center', ncol=5, bbox_to_anchor=(0.5, -0.02))
plt.tight_layout(rect=[0, 0.05, 1, 0.96])
plt.show()

In [ ]:
# Step 5: Model estimation - compare different specifications
print("Step 5: Model Comparison")
print("="*60)
print(f"{'Model':<20} {'AIC':>12} {'BIC':>12}")
print("-"*60)

models_to_try = [
    (0, 1, 0), (1, 1, 0), (0, 1, 1), 
    (1, 1, 1), (2, 1, 0), (0, 1, 2), 
    (2, 1, 1), (1, 1, 2)
]

results_comparison = {}
for order in models_to_try:
    try:
        model = ARIMA(y_train, order=order)
        res = model.fit()
        results_comparison[order] = {'aic': res.aic, 'bic': res.bic, 'model': res}
        print(f"ARIMA{order:<13} {res.aic:>12.2f} {res.bic:>12.2f}")
    except Exception as e:
        pass

# Find best model
best_aic = min(results_comparison.items(), key=lambda x: x[1]['aic'])
best_bic = min(results_comparison.items(), key=lambda x: x[1]['bic'])

print("-"*60)
print(f"Best by AIC: ARIMA{best_aic[0]}")
print(f"Best by BIC: ARIMA{best_bic[0]}")

In [ ]:
# Step 7: Forecasting
forecast_steps = len(y_test)
forecast = best_model.get_forecast(steps=forecast_steps)
forecast_mean = forecast.predicted_mean
forecast_ci = forecast.conf_int()

# Handle both DataFrame and array formats
if hasattr(forecast_ci, 'iloc'):
    ci_lower = forecast_ci.iloc[:, 0]
    ci_upper = forecast_ci.iloc[:, 1]
else:
    ci_lower = forecast_ci[:, 0]
    ci_upper = forecast_ci[:, 1]

# Plot
fig, ax = plt.subplots(figsize=(14, 6))

# Training data (last 100 points)
plot_start = max(0, train_size - 100)
ax.plot(range(plot_start, train_size), y_train[plot_start:], color=COLORS['blue'], linewidth=1.5, label='Training')

# Test data (actual)
test_index = range(train_size, len(y_full))
ax.plot(test_index, y_test, color=COLORS['orange'], linewidth=1.5, label='Actual')

# Forecast
ax.plot(test_index, forecast_mean, color=COLORS['red'], linewidth=2, linestyle='--', label='Forecast')
ax.fill_between(test_index, ci_lower, ci_upper, color=COLORS['red'], alpha=0.2, label='95% CI')

ax.axvline(x=train_size, color='black', linestyle='-', alpha=0.3)
ax.set_title(f'Step 7: Forecasting Industrial Production with ARIMA{best_order}', fontweight='bold')
ax.set_xlabel('Time')
ax.set_ylabel('Index')

# Get handles and labels for fig.legend
handles, labels = ax.get_legend_handles_labels()

plt.subplots_adjust(bottom=0.15)
fig.legend(handles, labels, loc='lower center', ncol=4, bbox_to_anchor=(0.5, -0.02))
plt.tight_layout(rect=[0, 0.08, 1, 1])
plt.show()

In [ ]:
# Fit the best model and show summary
best_order = best_aic[0]
best_model = ARIMA(y_train, order=best_order).fit()

print(f"\nStep 5: Fitting ARIMA{best_order}")
print("="*60)
print(best_model.summary())

In [ ]:
# Step 6: Diagnostic checking
residuals = best_model.resid

fig, axes = plt.subplots(2, 2, figsize=(14, 8))
fig.suptitle('Step 6: Residual Diagnostics', fontsize=14, fontweight='bold')

# Residuals over time
axes[0, 0].plot(residuals, color=COLORS['blue'], linewidth=0.5, label='Residuals')
axes[0, 0].axhline(y=0, color='red', linestyle='--')
axes[0, 0].set_title('Residuals Over Time', fontweight='bold')
axes[0, 0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.12), frameon=False)

# Histogram
axes[0, 1].hist(residuals, bins=30, color=COLORS['blue'], edgecolor='black', 
                alpha=0.7, density=True, label='Residuals')
x = np.linspace(residuals.min(), residuals.max(), 100)
axes[0, 1].plot(x, stats.norm.pdf(x, residuals.mean(), residuals.std()), 
                color=COLORS['red'], linewidth=2, label='Normal')
axes[0, 1].set_title('Residual Distribution', fontweight='bold')
axes[0, 1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.12), ncol=2, frameon=False)

# ACF of residuals
plot_acf(residuals, ax=axes[1, 0], lags=20, color=COLORS['blue'])
axes[1, 0].set_title('ACF of Residuals (should be white noise)', fontweight='bold')

# Q-Q plot
(osm, osr), (slope, intercept, r) = stats.probplot(residuals, dist="norm")
axes[1, 1].scatter(osm, osr, color=COLORS['blue'], s=20, alpha=0.5, label='Sample')
axes[1, 1].plot(osm, slope*osm + intercept, color=COLORS['red'], linewidth=2, label='Theoretical')
axes[1, 1].set_title('Q-Q Plot', fontweight='bold')
axes[1, 1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.12), ncol=2, frameon=False)

plt.tight_layout()
plt.show()

In [ ]:
# Download EUR/USD data
eurusd = yf.download('EURUSD=X', start='2020-01-01', end='2024-12-31', progress=False)

if isinstance(eurusd.columns, pd.MultiIndex):
    eurusd.columns = eurusd.columns.droplevel(1)

fx = eurusd['Close'].dropna()
print(f"EUR/USD Data: {len(fx)} observations")

# Plot
fig, ax = plt.subplots(figsize=(14, 5))
ax.plot(fx.index, fx.values, color=COLORS['blue'], linewidth=1, label='EUR/USD')
ax.set_title('EUR/USD Exchange Rate', fontweight='bold')
ax.set_ylabel('Exchange Rate')

# Get handles and labels for fig.legend
handles, labels = ax.get_legend_handles_labels()

plt.subplots_adjust(bottom=0.15)
fig.legend(handles, labels, loc='lower center', ncol=1, bbox_to_anchor=(0.5, -0.02))
plt.tight_layout(rect=[0, 0.08, 1, 1])
plt.show()

In [ ]:
# Step 7: Forecasting
forecast_steps = len(y_test)
forecast = best_model.get_forecast(steps=forecast_steps)
forecast_mean = forecast.predicted_mean
forecast_ci = forecast.conf_int()

# Handle both DataFrame and array formats
if hasattr(forecast_ci, 'iloc'):
    ci_lower = forecast_ci.iloc[:, 0]
    ci_upper = forecast_ci.iloc[:, 1]
else:
    ci_lower = forecast_ci[:, 0]
    ci_upper = forecast_ci[:, 1]

# Plot
fig, ax = plt.subplots(figsize=(14, 6))

# Training data (last 100 points)
plot_start = max(0, train_size - 100)
ax.plot(range(plot_start, train_size), y_train[plot_start:], color=COLORS['blue'], linewidth=1.5, label='Training')

# Test data (actual)
test_index = range(train_size, len(y_full))
ax.plot(test_index, y_test, color=COLORS['orange'], linewidth=1.5, label='Actual')

# Forecast
ax.plot(test_index, forecast_mean, color=COLORS['red'], linewidth=2, linestyle='--', label='Forecast')
ax.fill_between(test_index, ci_lower, ci_upper, color=COLORS['red'], alpha=0.2, label='95% CI')

ax.axvline(x=train_size, color='black', linestyle='-', alpha=0.3)
ax.set_title(f'Step 7: Forecasting Industrial Production with ARIMA{best_order}', fontweight='bold')
ax.set_xlabel('Time')
ax.set_ylabel('Index')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=4, frameon=False)
plt.tight_layout()
plt.show()

In [ ]:
# Forecast accuracy metrics
forecast_array = np.array(forecast_mean)
actual_array = np.array(y_test)

mae = np.mean(np.abs(forecast_array - actual_array))
rmse = np.sqrt(np.mean((forecast_array - actual_array)**2))
mape = np.mean(np.abs((actual_array - forecast_array) / actual_array)) * 100

print("Forecast Accuracy Metrics:")
print("="*40)
print(f"MAE:  {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAPE: {mape:.2f}%")

In [ ]:
# Forecast
fc, conf = fx_auto.predict(n_periods=len(fx_test), return_conf_int=True)

# Plot
fig, ax = plt.subplots(figsize=(14, 5))

ax.plot(fx.index[:train_end][-100:], fx_train[-100:], color=COLORS['blue'], linewidth=1.5, label='Training')
ax.plot(fx.index[train_end:], fx_test, color=COLORS['orange'], linewidth=1.5, label='Actual')
ax.plot(fx.index[train_end:], fc, color=COLORS['red'], linewidth=2, linestyle='--', label='Forecast')
ax.fill_between(fx.index[train_end:], conf[:, 0], conf[:, 1], color=COLORS['red'], alpha=0.2, label='95% CI')

ax.axvline(x=fx.index[train_end], color='black', linestyle='-', alpha=0.3)
ax.set_title(f'EUR/USD Forecast with ARIMA{fx_auto.order}', fontweight='bold')

# Get handles and labels for fig.legend
handles, labels = ax.get_legend_handles_labels()

plt.subplots_adjust(bottom=0.15)
fig.legend(handles, labels, loc='lower center', ncol=4, bbox_to_anchor=(0.5, -0.02))
plt.tight_layout(rect=[0, 0.08, 1, 1])
plt.show()

In [ ]:
# Accuracy
rmse = np.sqrt(np.mean((fc - fx_test)**2))
print(f"\nForecast RMSE: {rmse:.6f}")

In [ ]:
# Download EUR/USD data
eurusd = yf.download('EURUSD=X', start='2020-01-01', end='2024-12-31', progress=False)

if isinstance(eurusd.columns, pd.MultiIndex):
    eurusd.columns = eurusd.columns.droplevel(1)

fx = eurusd['Close'].dropna()
print(f"EUR/USD Data: {len(fx)} observations")

# Plot
fig, ax = plt.subplots(figsize=(14, 5))
ax.plot(fx.index, fx.values, color=COLORS['blue'], linewidth=1, label='EUR/USD')
ax.set_title('EUR/USD Exchange Rate', fontweight='bold')
ax.set_ylabel('Exchange Rate')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), frameon=False)
plt.tight_layout()
plt.show()

In [ ]:
# Unit root test
comprehensive_unit_root_test(fx, "EUR/USD")

In [ ]:
# Auto ARIMA
fx_values = fx.values
train_end = len(fx_values) - 50

fx_train = fx_values[:train_end]
fx_test = fx_values[train_end:]

fx_auto = pm.auto_arima(
    fx_train,
    start_p=0, start_q=0,
    max_p=3, max_q=3,
    d=None,
    seasonal=False,
    stepwise=True,
    suppress_warnings=True,
    trace=True
)

print(f"\nSelected model: ARIMA{fx_auto.order}")

In [ ]:
# Forecast
fc, conf = fx_auto.predict(n_periods=len(fx_test), return_conf_int=True)

# Plot
fig, ax = plt.subplots(figsize=(14, 5))

ax.plot(fx.index[:train_end][-100:], fx_train[-100:], color=COLORS['blue'], linewidth=1.5, label='Training')
ax.plot(fx.index[train_end:], fx_test, color=COLORS['orange'], linewidth=1.5, label='Actual')
ax.plot(fx.index[train_end:], fc, color=COLORS['red'], linewidth=2, linestyle='--', label='Forecast')
ax.fill_between(fx.index[train_end:], conf[:, 0], conf[:, 1], color=COLORS['red'], alpha=0.2, label='95% CI')

ax.axvline(x=fx.index[train_end], color='black', linestyle='-', alpha=0.3)
ax.set_title(f'EUR/USD Forecast with ARIMA{fx_auto.order}', fontweight='bold')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=4, frameon=False)
plt.tight_layout()
plt.show()

# Accuracy
rmse = np.sqrt(np.mean((fc - fx_test)**2))
print(f"\nForecast RMSE: {rmse:.6f}")

## Exercise 5: Practice Questions

Answer the following questions based on your analysis:

1. **Why do we test for unit roots before fitting an ARIMA model?**

2. **What happens if we fit an ARMA model to non-stationary data?**

3. **How do you interpret an ADF p-value of 0.15?**

4. **If KPSS rejects but ADF doesn't reject, what does this suggest?**

5. **Why do ARIMA forecast intervals grow over time?**

## Summary

### What We Practiced

1. **Unit Root Testing**: ADF and KPSS tests with real data
2. **Box-Jenkins Methodology**: Complete workflow from identification to forecasting
3. **Model Selection**: Using AIC, BIC, and auto_arima
4. **Diagnostics**: Residual analysis, Ljung-Box test
5. **Forecasting**: Point forecasts and confidence intervals

### Key Takeaways

- Stock prices are typically I(1), returns are I(0)
- Use both ADF and KPSS for robust conclusions
- ARIMA forecasts for I(1) series have growing uncertainty
- Always validate model with residual diagnostics